In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.interpolate import PchipInterpolator
from scipy.integrate import quad
from scipy.stats import bootstrap
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Level-1 Trigger (CMS)

In [2]:
egamma = pd.read_csv("l1t_data/egamma.csv")
tau = pd.read_csv("l1t_data/isolated tau.csv")
jet_energy = pd.read_csv("l1t_data/jet energy.csv")
muon = pd.read_csv("l1t_data/single muon.csv")

In [3]:
rates = pd.read_excel("l1t_data/trigger_rates.xlsx")

In [4]:
triggers = ["Jet", "Muon", "EGamma", "Tau"]
rejection_ratio = 400
jet_rate = rates["Jet Rate"][0] / rejection_ratio
muon_rate = rates["Muon Rate"][0] / rejection_ratio
egamma_rate = rates["Egamma Rate"][0] / rejection_ratio
tau_rate = rates["Tau Rate"][0] / rejection_ratio
all_rates = [jet_rate, muon_rate, egamma_rate, tau_rate]

In [5]:
fig = go.Figure()
fig.add_bar(x = triggers, y = all_rates)
fig.update_layout(width = 800, height = 600)

In [6]:
exp_dist = lambda x, l: l * np.exp(-1 * l * x) * (x > 0)

In [7]:
exp_cdf = lambda x, l: 1 - np.exp(-l * x)

In [8]:
exp_generator = lambda p, l: (-1 / l) * np.log(1 - p)

In [9]:
xs = np.linspace(0.01, 5.0, 101)
fig = go.Figure()
fig.add_scatter(x = xs, y = exp_dist(xs, 1.5), name = "PDF")
fig.add_scatter(x = xs, y = exp_cdf(xs, 1.5), name = "CDF")
fig.update_layout(width = 800, height = 600)

In [10]:
data_range = lambda x: np.linspace(np.min(x), np.max(x), 101)

In [11]:
egamma_spl = PchipInterpolator(egamma["x"], egamma[" y"])

In [12]:
egamma_xs = data_range(egamma["x"])

In [13]:
egamma_rate

0.00111

# Fit to EGamma L1T

In [14]:
#fit to muon - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
egamma_fit = lambda l: np.abs(egamma_rate - quad(lambda x: exp_dist(x, l) * egamma_spl(x), np.min(egamma_xs), np.max(egamma_xs))[0])

In [15]:
egamma_soln = minimize_scalar(egamma_fit, bounds = [0.00, 0.40], method="bounded")
egamma_l = egamma_soln.x

In [16]:
egamma_soln

     fun: 1.1000603183149854e-08
 message: 'Solution found.'
    nfev: 15
     nit: 15
  status: 0
 success: True
       x: 0.3091679635268055

In [17]:
egamma_threshold = 30
egamma_ideal = lambda x: 1.0 * (x > egamma_threshold)

In [18]:
fig = go.Figure()
fig.add_scatter(x = egamma["x"], y = egamma[" y"], name="Data")
fig.add_scatter(x = egamma_xs, y = egamma_spl(egamma_xs), name="Interpolation")
fig.add_scatter(x = egamma_xs, y = egamma_ideal(egamma_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = egamma_xs, y = 1.0 - exp_cdf(egamma_xs, egamma_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "e-gamma L1T efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )
fig.update_xaxes(type = "log")

In [19]:
fig = go.Figure()
fig.add_scatter(x = egamma["x"], y = egamma[" y"], name="Data")
fig.add_scatter(x = egamma_xs, y = egamma_spl(egamma_xs), name="Interpolation")
fig.add_scatter(x = egamma_xs, y = egamma_ideal(egamma_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = egamma_xs, y = 1.0 - exp_cdf(egamma_xs, egamma_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "e-gamma L1T efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )


## Fit to Tau 

In [20]:
tau_xs = data_range(tau["x"])
tau_spl = PchipInterpolator(tau["x"], tau[" y"])

In [21]:
#fit to tau - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
tau_fit = lambda l: np.abs(tau_rate - quad(lambda x: exp_dist(x, l) * tau_spl(x), 0, 100)[0])

In [22]:
tau_threshold = 38
tau_ideal = lambda x: 1.0 * (x > tau_threshold)

In [23]:
tau_soln = minimize_scalar(tau_fit)
tau_l = tau_soln.x

In [24]:
tau_soln

     fun: 6.245224172876651e-12
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 30
     nit: 26
 success: True
       x: 0.4770168907807679

In [25]:
tau_l

0.4770168907807679

In [26]:
fig = go.Figure()
fig.add_scatter(x = tau["x"], y = tau[" y"], name = "Data")
fig.add_scatter(x = tau_xs, y = tau_spl(tau_xs), name = "Interpolation")
fig.add_scatter(x = tau_xs, y = tau_ideal(tau_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = tau_xs, y = 1.0 - exp_cdf(tau_xs, tau_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Tau L1T Efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )
fig.update_xaxes(type = "linear")

## Fit to Jets

In [27]:
jet_xs = data_range(jet_energy["x"])
jet_spl = PchipInterpolator(jet_energy["x"], jet_energy[" y"])

In [28]:
jet_fit = lambda l: np.abs(jet_rate - quad(lambda x: exp_dist(x, l) * jet_spl(x), 0, np.max(jet_xs))[0])
jet_soln = minimize_scalar(jet_fit, bounds = [0.000, 0.40], method="bounded")
jet_l = jet_soln.x

In [29]:
jet_soln

     fun: 2.4308430632432286e-07
 message: 'Solution found.'
    nfev: 18
     nit: 18
  status: 0
 success: True
       x: 0.02749101013719685

In [30]:
jet_l

0.02749101013719685

In [31]:
jet_threshold = 320
jet_ideal = lambda x: 1.0 * (x > jet_threshold)

In [32]:
fig = go.Figure()
fig.add_scatter(x = jet_energy["x"], y = jet_energy[" y"], name = "Data")
fig.add_scatter(x = jet_xs, y = jet_spl(jet_xs), name = "Interpolation")
fig.add_scatter(x = jet_xs, y = jet_ideal(jet_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = jet_xs, y = 1.0 - exp_cdf(jet_xs, jet_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Jet Energy L1T Efficiency",
                  xaxis_title = "Momentum (GeV)"
                  )
fig.update_xaxes(type = "linear")

## Fit to Muon

In [33]:
muon_xs = data_range(muon["x"])
muon_spl = PchipInterpolator(muon["x"], muon[" y"])

In [34]:
#fit to muon - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
muon_fit = lambda l: np.abs(muon_rate - quad(lambda x: exp_dist(x, l) * muon_spl(x), 0, np.max(muon_xs))[0])
muon_soln = minimize_scalar(muon_fit, bounds = [0.00, 0.60], method="bounded")
muon_l = muon_soln.x

In [35]:
muon_soln

     fun: 0.005345849586570096
 message: 'Solution found.'
    nfev: 14
     nit: 14
  status: 0
 success: True
       x: 0.5655829287920927

In [36]:
muon_l

0.5655829287920927

In [37]:
muon_threshold = 22
muon_ideal = lambda x: 1.0 * (x > muon_threshold)

In [38]:
fig = go.Figure()
fig.add_scatter(x = muon["x"], y = muon[" y"], name = "data")
fig.add_scatter(x = muon_xs, y = muon_spl(muon_xs), name = "Interpolation")
fig.add_scatter(x = muon_xs, y = muon_ideal(muon_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = muon_xs, y = 1.0 - exp_cdf(muon_xs, muon_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Single Muon L1T Efficiency",
                  xaxis_title = "Momentum (GeV)"
                  )
fig.update_xaxes(type = "linear")

# Modeling Skill

In [39]:
#find the percentile of measurements above threshold
egamma_prctile = exp_cdf(egamma_threshold, egamma_l)
jet_prctile = exp_cdf(jet_threshold, jet_l)
muon_prctile = exp_cdf(muon_threshold, muon_l)
tau_prctile = exp_cdf(tau_threshold, tau_l) 

In [40]:
thresholds = np.array([jet_threshold, muon_threshold, egamma_threshold, tau_threshold])

In [41]:
prctiles = np.array([jet_prctile, muon_prctile, egamma_prctile, tau_prctile])

In [42]:
prctiles

array([0.99984883, 0.99999605, 0.99990626, 0.99999999])

In [43]:
def generate_null():
    p = np.random.uniform(size=(4)) * prctiles
    jet = exp_generator(p[0], jet_l)
    muon = exp_generator(p[1], muon_l)
    egamma = exp_generator(p[2], egamma_l)
    tau = exp_generator(p[3], tau_l)
    

    e = np.array([jet, muon, egamma, tau])
    z = np.array([jet_spl(e[0]),
                muon_spl(e[1]),
                egamma_spl(e[2]),
                tau_spl(e[3])])
    
    res = np.stack((e, z))
    
    return res


In [44]:
null_evt = generate_null()

In [45]:
n_samples = int(1e6)

In [46]:
null_evts = np.stack([generate_null() for i in range(n_samples)])

In [47]:
null_evts.shape

(1000000, 2, 4)

In [48]:
null_z = np.sum(null_evts, axis=2)[:,1]

In [49]:
np.std(null_z)

0.015944719864178607

In [50]:
triggers

['Jet', 'Muon', 'EGamma', 'Tau']

In [51]:
prctiles

array([0.99984883, 0.99999605, 0.99990626, 0.99999999])

In [52]:
evt_rng = np.random.default_rng()

In [53]:
trigger_rates = rates["Proportion"]

In [54]:
trigger_choice = lambda: evt_rng.choice(np.arange(len(trigger_rates)), 1, p = trigger_rates)[0]

In [55]:
rates.iloc[trigger_choice()]

Percentage            24.800
Proportion             0.248
Jet / Jet energy       0.000
Muon                   0.000
Egamma                 1.000
Tau                    0.000
Jet proportions        0.000
Muon proportions       0.000
Egamma proportions     0.248
Tau proportions        0.000
Jet Rate                 NaN
Muon Rate                NaN
Egamma Rate              NaN
Tau Rate                 NaN
Name: 11, dtype: float64

In [56]:
def generate_positive():
    #select a trigger outcome with rates proportional to those recorded
    outcome_idx = trigger_choice()
    outcome = rates.iloc[outcome_idx]

    #determine the triggers for that outcome
    trig_egamma = outcome["Egamma"]
    trig_jet = outcome["Jet / Jet energy"]
    trig_muon = outcome["Muon"]
    trig_tau = outcome["Tau"]

    triggers = np.array([trig_jet, trig_muon, trig_egamma, trig_tau])

    #generate particle energies from the distributions based on whether or not they're above trigger levels
    def transform_p(prctile, outcome):
        p = np.random.uniform()
        if outcome == 1.0:
            #generate an event above the trigger threshold
            return p * (1 - prctile) + prctile
        else:
            #generate an event below the trigger threshold
            return p * prctile
        
    p_jet = transform_p(jet_prctile, trig_jet)
    p_muon = transform_p(muon_prctile, trig_muon)
    p_egamma = transform_p(egamma_prctile, trig_egamma)
    p_tau = transform_p(tau_prctile, trig_tau)

    jet = exp_generator(p_jet, jet_l)
    muon = exp_generator(p_muon, muon_l)
    egamma = exp_generator(p_egamma, egamma_l)
    tau = exp_generator(p_tau, tau_l)

    e = np.array([jet, muon, egamma,  tau,])
    z = np.array([jet_spl(e[0]),
                  muon_spl(e[1]),
                  egamma_spl(e[2]),
                  tau_spl(e[3]),
                  ])
    
    res = np.stack((e, z))
    
    return res


In [57]:
pos_evt = generate_positive()

In [58]:
pos_evt

array([[9.13743499e+01, 1.26628413e+00, 3.02212794e+01, 7.88039578e-01],
       [0.00000000e+00, 5.11336581e-03, 4.29933587e-01, 1.69149466e-05]])

In [59]:
thresholds

array([320,  22,  30,  38])

In [60]:
pos_evt[1,:]

array([0.00000000e+00, 5.11336581e-03, 4.29933587e-01, 1.69149466e-05])

In [61]:
pos_evts = np.stack([generate_positive() for i in range(n_samples)])

In [62]:
pos_z = np.sum(pos_evts, axis=2)[:,1]

## determine statistics

In [63]:
from sklearn.svm import LinearSVC

In [64]:
classifier = LinearSVC()

In [65]:
x = np.concatenate([null_z, pos_z]).reshape(-1,1)

In [66]:
y = np.concatenate([np.zeros(null_z.shape), np.ones(pos_z.shape)])

In [67]:
classifier.fit(x, y)

LinearSVC()

In [68]:
null_z2 = classifier.decision_function(null_z.reshape(-1,1))

In [69]:
pred_null = classifier.predict(null_z.reshape(-1,1))

In [70]:
pos_z2 = classifier.decision_function(pos_z.reshape(-1,1))

In [71]:
pred_pos = classifier.predict(pos_z.reshape(-1,1))

In [72]:
classifier.score(x, y)

0.9996055

In [73]:
confusion = [[np.mean(pred_null == 0), np.mean(pred_null == 1),],
             [np.mean(pred_pos == 0), np.mean(pred_pos == 1)],]

In [74]:
confusion

[[0.999211, 0.000789], [0.0, 1.0]]